In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# =============================================
# Load and Prepare the Gas Price Data
# =============================================

file_path = "/content/Nat_Gas.csv"
df = pd.read_csv(file_path)

# Normalize columns
df.columns = [c.strip().lower() for c in df.columns]
date_col = [c for c in df.columns if 'date' in c or 'month' in c][0]
price_col = [c for c in df.columns if 'price' in c][0]

df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col)
df.set_index(date_col, inplace=True)

# Fit ARIMA for forecasting
model = SARIMAX(df[price_col],
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False)
results = model.fit(disp=False)

# =============================================
# Helper: Get estimated price for any date
# =============================================
def estimate_price(date_str):
    """Estimate price for a given date using ARIMA model."""
    input_date = pd.to_datetime(date_str)
    last_date = df.index[-1]

    if input_date <= last_date:
        closest_date = df.index[df.index.get_indexer([input_date], method='nearest')[0]]
        return df.loc[closest_date, price_col]
    else:
        months_ahead = (input_date.year - last_date.year) * 12 + (input_date.month - last_date.month)
        forecast_future = results.get_forecast(steps=months_ahead)
        return forecast_future.predicted_mean.iloc[-1]

# =============================================
# Main Function: Contract Valuation
# =============================================

def price_storage_contract(injection_dates, withdrawal_dates,
                           inject_rate, withdraw_rate,
                           max_volume, storage_cost_per_month):
    """
    Calculate total value of a natural gas storage contract.

    Parameters:
        injection_dates (list): list of injection date strings (YYYY-MM-DD)
        withdrawal_dates (list): list of withdrawal date strings (YYYY-MM-DD)
        inject_rate (float): volume injected per injection date (in MMBtu)
        withdraw_rate (float): volume withdrawn per withdrawal date (in MMBtu)
        max_volume (float): maximum storage capacity (in MMBtu)
        storage_cost_per_month (float): monthly storage fee in $
    """
    storage_volume = 0.0
    total_cost = 0.0
    total_revenue = 0.0
    months = set()

    # Handle injections
    for date in injection_dates:
        price = estimate_price(date)
        if storage_volume + inject_rate > max_volume:
            inject_rate = max_volume - storage_volume  # limit to capacity
        storage_volume += inject_rate
        total_cost += inject_rate * price
        months.add(pd.to_datetime(date).strftime("%Y-%m"))
        print(f"Injected {inject_rate} MMBtu @ ${price:.2f} on {date}")

    # Handle withdrawals
    for date in withdrawal_dates:
        price = estimate_price(date)
        if storage_volume < withdraw_rate:
            withdraw_rate = storage_volume  # withdraw only what’s available
        storage_volume -= withdraw_rate
        total_revenue += withdraw_rate * price
        months.add(pd.to_datetime(date).strftime("%Y-%m"))
        print(f"Withdrew {withdraw_rate} MMBtu @ ${price:.2f} on {date}")

    # Storage cost (assume charged monthly across contract period)
    total_storage_cost = len(months) * storage_cost_per_month
    print(f"Storage cost for {len(months)} months = ${total_storage_cost:,.2f}")

    # Final contract value
    contract_value = total_revenue - total_cost - total_storage_cost
    print(f"\nTotal Revenue: ${total_revenue:,.2f}")
    print(f"Total Cost:    ${total_cost:,.2f}")
    print(f"Storage Fees:  ${total_storage_cost:,.2f}")
    print(f"--------------------------------")
    print(f"Contract Value: ${contract_value:,.2f}")

    return contract_value


# =============================================
# Example Test
# =============================================

injection_dates = ["2023-05-31", "2023-06-30"]
withdrawal_dates = ["2023-10-31", "2023-11-30"]

value = price_storage_contract(
    injection_dates=injection_dates,
    withdrawal_dates=withdrawal_dates,
    inject_rate=500000,          # 500K MMBtu injected per month
    withdraw_rate=500000,        # withdrawn per month
    max_volume=1_000_000,        # max capacity 1 million MMBtu
    storage_cost_per_month=100000  # $100K per month
)

print(f"\nEstimated contract value: ${value:,.2f}")


/tmp/ipython-input-2391579707.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Injected 500000 MMBtu @ $11.20 on 2023-05-31
Injected 500000 MMBtu @ $10.90 on 2023-06-30
Withdrew 500000 MMBtu @ $11.80 on 2023-10-31
Withdrew 500000 MMBtu @ $12.20 on 2023-11-30
Storage cost for 4 months = $400,000.00

Total Revenue: $12,000,000.00
Total Cost:    $11,050,000.00
Storage Fees:  $400,000.00
--------------------------------
Contract Value: $550,000.00

Estimated contract value: $550,000.00
